<a href="https://colab.research.google.com/github/emely3h/Geospatial_ML/blob/feature%2Fphysics-jaccard-index/data_exploration/physics_jaccard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Physics Jaccard Index

Calculate the physics jaccard index which will be the main success metric.

# Small Subset(256x256: Training)

### 0. Get Stats for each image

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
! ls
%cd drive/MyDrive/MachineLearning/Geospatial_ML
! ls

drive  sample_data
/content/drive/.shortcut-targets-by-id/15HUD3sGdfvxy5Y_bjvuXgrzwxt7TzRfm/MachineLearning/Geospatial_ML
architecture.drawio  experiment_1_2.ipynb  prepare_data
data_exploration     experiments	   README.md
evaluation	     models		   requirements.txt


In [ ]:
! git checkout feature/physics-jaccard-index
! git pull

Already on 'feature/physics-jaccard-index'
Your branch is up to date with 'origin/feature/physics-jaccard-index'.
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 4 (delta 2), reused 4 (delta 2), pack-reused 0
Unpacking objects: 100% (4/4), 676 bytes | 2.00 KiB/s, done.
From https://github.com/emely3h/Geospatial_ML
   7f09b81..06a6ecc  feature/physics-jaccard-index -> origin/feature/physics-jaccard-index
Updating 7f09b81..06a6ecc
Fast-forward
 models/helpers.py | 19 +++++++++++++++++++
 1 file changed, 19 insertions(+)


In [4]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from data_exploration.mask_stats import Mask_Stats
from prepare_data.create_mask import create_physical_mask
from tensorflow.keras.utils import to_categorical
from tensorflow import keras
from models.unet_model import unet_2d
# from models.helpers import copy_data_to_arrays, save_pickle, jaccard_coef

### 1. Loading + Preparing Training Data

In [5]:
total_tiles = 11121
train_tiles = total_tiles // 100 * 60 +1
test_val_tiles = total_tiles // 100 * 20 +1
data_path = "../data_colab/256_256"

train_split_x = np.memmap(os.path.join(data_path, "train_split_x.npy"), mode="r", shape=(train_tiles, 256, 256, 5), dtype=np.float32)
train_split_y = np.memmap(os.path.join(data_path, "train_split_y.npy"), mode="r", shape=(train_tiles, 256, 256), dtype=np.float32)

train_stats = Mask_Stats(train_split_y)
train_stats.print_stats()


Shape: (6661, 256, 256)
Land pixels: 176919986  40.528 %
Valid pixels: 125877821  28.836 %
Invalid pixels: 133737489  30.636 %
Sum: 6661


In [13]:
print(int(np.ceil(6661/1000)))

7


### 2. Initializing training data

In [7]:
import numpy as np
from typing import Tuple
from skimage.measure import label
from keras.utils import to_categorical
from tensorflow import keras

class JaccardIndexCalculator:

    def __init__(self, split_x: np.ndarray, split_y: np.ndarray, tiles: int, chunk_size: int):
        self.split_x = split_x
        self.split_y = split_y
        self.tiles = tiles
        self.chunk_size = chunk_size
        self.num_chunks = int(np.ceil(tiles/chunk_size))
        self.current_chunk_index = 0
        self.jaccard_indices = []

    def __iter__(self):
        return self

    def __next__(self):
        if self.current_chunk_index >= self.num_chunks:
            raise StopIteration

        x_input_chunk, y_mask_chunk = self.copy_data_to_array()
        pred_physical = self.create_physical_mask(x_input_chunk)
        y_one_hot = to_categorical(y_mask_chunk, num_classes=3)
        print(f"Y one hot No.{self.current_chunk_index} Done")
        jaccard_index = self.jaccard_coef(pred_physical[:, :, :, 1], y_one_hot[:, :, :, 1])
        print(f"Jaccard_index:{jaccard_index}")
        print(f"Jaccard_index No.{self.current_chunk_index} Done")
        self.jaccard_indices.append(jaccard_index)
        print(f"Chunk No.{self.current_chunk_index} Done")
        self.current_chunk_index += 1
        return jaccard_index

    def copy_data_to_array(self) -> Tuple[np.ndarray, np.ndarray]:
      start_index = self.current_chunk_index * self.chunk_size
      end_index = start_index + self.chunk_size
      if end_index > self.tiles:
          end_index = self.tiles
      chunk_size = end_index - start_index
      print("chunk_size:", chunk_size)
      x_input = np.zeros((chunk_size, 256, 256, 5), dtype=np.float32)
      np.copyto(x_input, self.split_x[start_index:end_index])
      y_mask = np.zeros((chunk_size, 256, 256), dtype=np.float32)
      np.copyto(y_mask, self.split_y[start_index:end_index])
      print(f"Copy No.{self.current_chunk_index} Done")
      return x_input, y_mask


    def label_pixels(self, img):
        mask1 = img == 0
        mask2 = (img == 255) | (img == 253)
        img[mask1] = 2
        img[mask2] = 0
        img[~(mask1 | mask2)] = 1
        return img

    def create_physical_mask(self, x_input: np.ndarray) -> np.ndarray:
      wq_channel = x_input[:, :, :, 4]
      labeled = self.label_pixels(wq_channel)
      print(f"Create a Physical Mask No.{self.current_chunk_index} Done")
      return to_categorical(labeled, num_classes=3)

    def jaccard_coef(self, y_true: np.ndarray, y_pred: np.ndarray) -> keras.backend.floatx():
        y_true_f = keras.backend.flatten(y_true)
        y_pred_f = keras.backend.flatten(y_pred)

        intersection = keras.backend.sum(y_true_f * y_pred_f)
        return (intersection + 1.0) / (
            keras.backend.sum(y_true_f)
            + keras.backend.sum(y_pred_f)
            - intersection
            + 1.0
        )  # todo reason for +1?

    def calculate_mean_jaccard_index(self) -> keras.backend.floatx():
        for jaccard_index in self:
            pass
        return np.mean(self.jaccard_indices)


In [8]:
jaccard_index_calculator = JaccardIndexCalculator(train_split_x, train_split_y, train_tiles, chunk_size=1000)
mean_jaccard_index = jaccard_index_calculator.calculate_mean_jaccard_index()
print(mean_jaccard_index)

chunk_size: 1000
Copy No.0 Done
Create a Physical Mask No.0 Done
Y one hot No.0 Done
Jaccard_index:0.8042755722999573
Jaccard_index No.0 Done
Chunk No.0 Done
chunk_size: 1000
Copy No.1 Done
Create a Physical Mask No.1 Done
Y one hot No.1 Done
Jaccard_index:0.9152491092681885
Jaccard_index No.1 Done
Chunk No.1 Done
chunk_size: 1000
Copy No.2 Done
Create a Physical Mask No.2 Done
Y one hot No.2 Done
Jaccard_index:0.9058482050895691
Jaccard_index No.2 Done
Chunk No.2 Done
chunk_size: 1000
Copy No.3 Done
Create a Physical Mask No.3 Done
Y one hot No.3 Done
Jaccard_index:0.9375823140144348
Jaccard_index No.3 Done
Chunk No.3 Done
chunk_size: 1000
Copy No.4 Done
Create a Physical Mask No.4 Done
Y one hot No.4 Done
Jaccard_index:0.8414788246154785
Jaccard_index No.4 Done
Chunk No.4 Done
chunk_size: 1000
Copy No.5 Done
Create a Physical Mask No.5 Done
Y one hot No.5 Done
Jaccard_index:0.7649018168449402
Jaccard_index No.5 Done
Chunk No.5 Done
chunk_size: 661
Copy No.6 Done
Create a Physical Mas

In [ ]:
x_input,y_mask = copy_data_to_arrays(train_split_x, train_split_y, train_tiles, chunk_size)

Copying data to arrays...
x_input shape: (6661, 256, 256, 5)
x_input min value: 0.0 x_input max value: 0.0
Data copied to x_input...
x_input shape: (6661, 256, 256, 5)
x_input min value: 0.0 x_input max value: 255.0

Initializing y_mask...
y_mask shape: (6661, 256, 256)
y_mask min value: 0.0 y_mask max value: 0.0
Data copied to y_mask...
y_mask shape: (6661, 256, 256)
y_mask min value: 0.0 y_mask max value: 2.0


### 3. Create Physical mask(training data)

In [ ]:
pred_physical = create_physical_mask(x_input, chunk_size)

### 4. Encoding (training data)

In [ ]:
y_one_hot = to_categorical(y_mask, num_classes=3)

### 5. Calculate Physics jaccard index(training data)

In [ ]:
jaccard = jaccard_coef(y_one_hot, pred_physical)
print(jaccard)

tf.Tensor(0.7695766, shape=(), dtype=float32)


In [ ]:
# save_pickle(jaccard, "../data_colab/256_256", "train_physics_jaccard")

Saving training_physics_jaccard.pkl to ../data_colab/256_256...
Saved training_physics_jaccard.pkl to ../data_colab/256_256.


# All dataset(Overlap: 56px)

### 6. Loading all data(256_200)

In [ ]:
data_path = "../data_colab/256_200"
train_total_tiles = 11063
val_total_tiles = 3545
test_total_tiles = 3699

train_split_x = np.memmap(os.path.join(data_path, "train_split_x.npy"), mode="r", shape=(train_total_tiles, 256, 256, 5), dtype=np.uint8)
train_split_y = np.memmap(os.path.join(data_path, "train_split_y.npy"), mode="r", shape=(train_total_tiles, 256, 256), dtype=np.uint8)
val_split_x = np.memmap(os.path.join(data_path, "val_split_x.npy"), mode="r", shape=(val_total_tiles, 256, 256, 5), dtype=np.uint8)
val_split_y = np.memmap(os.path.join(data_path, "val_split_y.npy"), mode="r", shape=(val_total_tiles, 256, 256), dtype=np.uint8)
test_split_x = np.memmap(os.path.join(data_path, "test_split_x.npy"), mode="r", shape=(test_total_tiles, 256, 256, 5), dtype=np.uint8)
test_split_y = np.memmap(os.path.join(data_path, "test_split_y.npy"), mode="r", shape=(test_total_tiles, 256, 256), dtype=np.uint8)


### 7. Initializing all data(256_200)

In [ ]:
print("1: Training Dataset")
train_x_input, train_y_mask = copy_data_to_arrays(train_split_x, train_split_y, train_total_tiles, chunk_size)
print("\n2: Validation Dataset")
val_x_input, val_y_mask = copy_data_to_arrays(val_split_x, val_split_y, val_total_tiles, chunk_size)
print("\n3: Test Dataset")
test_x_input, test_y_mask = copy_data_to_arrays(test_split_x, test_split_y, test_total_tiles, chunk_size)

1: Training Dataset
Copying data to arrays...
x_input shape: (11063, 256, 256, 5)
x_input min value: 0.0 x_input max value: 0.0
Data copied to x_input...
x_input shape: (11063, 256, 256, 5)
x_input min value: 0.0 x_input max value: 255.0

Initializing y_mask...
y_mask shape: (11063, 256, 256)
y_mask min value: 0.0 y_mask max value: 0.0
Data copied to y_mask...
y_mask shape: (11063, 256, 256)
y_mask min value: 0.0 y_mask max value: 2.0

2: Validation Dataset
Copying data to arrays...
x_input shape: (3545, 256, 256, 5)
x_input min value: 0.0 x_input max value: 0.0
Data copied to x_input...
x_input shape: (3545, 256, 256, 5)
x_input min value: 0.0 x_input max value: 255.0

Initializing y_mask...
y_mask shape: (3545, 256, 256)
y_mask min value: 0.0 y_mask max value: 0.0
Data copied to y_mask...
y_mask shape: (3545, 256, 256)
y_mask min value: 0.0 y_mask max value: 2.0

3: Test Dataset
Copying data to arrays...
x_input shape: (3699, 256, 256, 5)
x_input min value: 0.0 x_input max value: 0.0

### 8. Create all Physical masks (256_200)

In [ ]:
train_pred_physical = create_physical_mask(train_x_input, chunk_size)
val_pred_physical = create_physical_mask(val_x_input, chunk_size)
test_pred_physical = create_physical_mask(test_x_input, chunk_size)

### 9. Encoding all data (256_200)

In [ ]:
train_y_one_hot = to_categorical(train_y_mask, num_classes=3)
val_y_one_hot = to_categorical(val_y_mask, num_classes=3)
test_y_one_hot = to_categorical(test_y_mask, num_classes=3)

### 10. Calculate all Physics jaccard index (256_200)

In [ ]:
train_jaccard = jaccard_coef(train_y_one_hot, train_pred_physical)
val_jaccard = jaccard_coef(val_y_one_hot, val_pred_physical)
test_jaccard = jaccard_coef(test_y_one_hot, test_pred_physical)

ResourceExhaustedError: ignored

In [ ]:
print("1.Traing dataset")
print(train_jaccard)
print("\n2.Validation dataset")
print(val_jaccard)
print("\n3.Test dataset")
print(test_jaccard)

In [ ]:
save_pickle(train_jaccard, "../data_colab/256_200", "train_physics_jaccard")
save_pickle(val_jaccard, "../data_colab/256_200", "val_physics_jaccard")
save_pickle(test_jaccard, "../data_colab/256_200", "test_physics_jaccard")

# All dataset(No overlap)

### 11. Loading all data(256_256)

In [ ]:
data_path = "../data_colab/256_256"
train_total_tiles = 11063
val_total_tiles = 3545
test_total_tiles = 3699

train_split_x = np.memmap(os.path.join(data_path, "train_split_x.npy"), mode="r", shape=(train_total_tiles, 256, 256, 5), dtype=np.uint8)
train_split_y = np.memmap(os.path.join(data_path, "train_split_y.npy"), mode="r", shape=(train_total_tiles, 256, 256), dtype=np.uint8)
val_split_x = np.memmap(os.path.join(data_path, "val_split_x.npy"), mode="r", shape=(val_total_tiles, 256, 256, 5), dtype=np.uint8)
val_split_y = np.memmap(os.path.join(data_path, "val_split_y.npy"), mode="r", shape=(val_total_tiles, 256, 256), dtype=np.uint8)
test_split_x = np.memmap(os.path.join(data_path, "test_split_x.npy"), mode="r", shape=(test_total_tiles, 256, 256, 5), dtype=np.uint8)
test_split_y = np.memmap(os.path.join(data_path, "test_split_y.npy"), mode="r", shape=(test_total_tiles, 256, 256), dtype=np.uint8)

### 12. Initializing all data(256_256)

In [ ]:
print("1: Training Dataset")
train_x_input, train_y_mask = initialize_saved_data(train_split_x, train_split_y, train_total_tiles)
print("\n2: Validation Dataset")
val_x_input, val_y_mask = initialize_saved_data(val_split_x, val_split_y, val_total_tiles)
print("\n3: Test Dataset")
test_x_input, test_y_mask = initialize_saved_data(test_split_x, test_split_y, test_total_tiles)

### 13. Create all Physical masks (256_256)

In [ ]:
train_pred_physical = create_physical_mask(train_x_input)
val_pred_physical = create_physical_mask(val_x_input)
test_pred_physical = create_physical_mask(test_x_input)

### 14. Encoding all data (256_256)

In [ ]:
train_y_one_hot = to_categorical(train_y_mask, num_classes=3)
val_y_one_hot = to_categorical(val_y_mask, num_classes=3)
test_y_one_hot = to_categorical(test_y_mask, num_classes=3)

### 15. Calculate all Physics jaccard index (256_256)

In [ ]:
train_jaccard = jaccard_coef(train_y_one_hot, train_pred_physical)
val_jaccard = jaccard_coef(val_y_one_hot, val_pred_physical)
test_jaccard = jaccard_coef(test_y_one_hot, test_pred_physical)

print("1.Traing dataset")
print(train_jaccard)
print("\n2.Validation dataset")
print(val_jaccard)
print("\n3.Test dataset")
print(test_jaccard)